In [1]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial

In [2]:
data=pd.read_csv("C:\\Users\\grsam\\Downloads\\kmeans_data\\data.csv")
labels = pd.read_csv("C:\\Users\\grsam\\Downloads\\kmeans_data\\label.csv",names=['label'],header=None)
data.head()                     
                 

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [4]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [5]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [6]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [7]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [8]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  26382.663917525773
Tolerance for the Iteration  1 :  6225.400808799114
Tolerance for the Iteration  2 :  5170.435170452872
Tolerance for the Iteration  3 :  3830.771234030605
Tolerance for the Iteration  4 :  2567.2366395996605
Tolerance for the Iteration  5 :  1985.3264446227931
Tolerance for the Iteration  6 :  1748.5673174157303
Tolerance for the Iteration  7 :  1366.127313218391
Tolerance for the Iteration  8 :  1014.4671474720157
Tolerance for the Iteration  9 :  909.5034748845326
Tolerance for the Iteration  10 :  924.902367027337
Tolerance for the Iteration  11 :  1013.6068072733923
Tolerance for the Iteration  12 :  862.3682487725041
Tolerance for the Iteration  13 :  797.9526508545518
Tolerance for the Iteration  14 :  696.5723945417743
Tolerance for the Iteration  15 :  537.4965950960501
Tolerance for the Iteration  16 :  539.2617405782978
Tolerance for the Iteration  17 :  440.73526093715316
Tolerance for the Iteration  18 :  526.68523219814

In [9]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [10]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25438313736.96912


In [11]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([0, 3, 2, 6, 1, 7, 9, 9, 3, 5])

In [12]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.0925

In [13]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  38494.174742268035
Tolerance for the Iteration  1 :  5032.886285172103
Tolerance for the Iteration  2 :  2703.4365757423075
Tolerance for the Iteration  3 :  4051.8601077377934
Tolerance for the Iteration  4 :  1226.9931622283107
Tolerance for the Iteration  5 :  1898.0997499749978
Tolerance for the Iteration  6 :  1089.0583156238472
Tolerance for the Iteration  7 :  1207.6141973982199
Tolerance for the Iteration  8 :  1025.5738113866635
Tolerance for the Iteration  9 :  1124.742692568226
Tolerance for the Iteration  10 :  2129.0731792337438
Tolerance for the Iteration  11 :  758.7086758675869
Tolerance for the Iteration  12 :  1230.2761425306776
Tolerance for the Iteration  13 :  0.0


In [14]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [15]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 2, 1, 2, 2, 7, 2, 2, 2, 2])

In [16]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [17]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  33060.25487841489
Tolerance for the Iteration  1 :  6229.789524262037
Tolerance for the Iteration  2 :  4358.892184441529
Tolerance for the Iteration  3 :  3445.1636552108594
Tolerance for the Iteration  4 :  2670.8742524296204
Tolerance for the Iteration  5 :  2472.438893135075
Tolerance for the Iteration  6 :  1288.528880986492
Tolerance for the Iteration  7 :  684.3535205088838
Tolerance for the Iteration  8 :  487.1901197604792
Tolerance for the Iteration  9 :  376.83042452830193
Tolerance for the Iteration  10 :  265.55106072074784
Tolerance for the Iteration  11 :  273.4816571138783
Tolerance for the Iteration  12 :  318.84250521298003
Tolerance for the Iteration  13 :  328.9036999304562
Tolerance for the Iteration  14 :  304.7661612130886
Tolerance for the Iteration  15 :  319.0663547613117
Tolerance for the Iteration  16 :  265.50686229905966
Tolerance for the Iteration  17 :  207.74428420016642
Tolerance for the Iteration  18 :  188.9125596184

In [18]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [19]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25438313736.96912
Jacard SSE: 34361687572.938736
Cosine SSE : 25417353508.869522


In [20]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([8, 7, 5, 9, 2, 1, 3, 0, 3, 0])

In [22]:
Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.0825

In [23]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.0925
Jacard accuracy: 0.1075
Cosine accuracy : 0.0825
